In [1]:
import numpy as np
import qpms
import warnings
from qpms.cybspec import BaseSpec
from qpms.cytmatrices import CTMatrix, TMatrixGenerator, TMatrixInterpolator
from qpms.qpms_c import Particle, pgsl_ignore_error
from qpms.cymaterials import EpsMu, EpsMuGenerator, LorentzDrudeModel, lorentz_drude
from qpms.cycommon import DebugFlags, dbgmsg_enable
from qpms import FinitePointGroup, ScatteringSystem, BesselType, eV, hbar
import scipy.constants as sci
eh = eV/hbar

In [2]:
#TODO
period = 520e-9
a1 = np.array([0,period])                                 
a2 = np.array([period,0])

In [3]:
#Particle positions
orig_x = [0]
orig_y = [0]
orig_xy = np.stack(np.meshgrid(orig_x,orig_y),axis=-1)

In [4]:
period = 0.52
refractive_index = 1.52 # for background medium
height = 50e-9 # Particle height
radius = 50e-9 # Particle radius
medium = EpsMu(refractive_index**2) # non-lossy background medium with constant refr. index #OK
# global symmetry group of the system
#sym = FinitePointGroup(point_group_info['D4h'])
omega = 1.58*eh
metal = lorentz_drude['Ag']
kx_lim = np.array([-0.2, 0.2], dtype=float)
N=501

In [5]:
omega_scuff_min=1.5*eh/3e14
omega_scuff_max=1.64*eh/3e14
omega_scuff=omega/3e14
omega_scuff_slr=(2*np.pi*sci.c/(1.52*0.52e-6))/3e14
omega_scuff_min, omega_scuff_max, omega_scuff_slr, omega_scuff

(7.59633723939313, 8.305328715069821, 7.94387469344152, 8.001475225494097)

In [6]:
bspec = BaseSpec(lMax = 1)

tmfile_scuffOld = '/home/javier/tmatrices/sphereAg_50nm_oldScuff.TMatrix'
tmfile_scuffNew = '/home/javier/tmatrices/sphere50nm_newScuff.TMatrix'
interp_old = TMatrixInterpolator(tmfile_scuffOld, bspec, atol=1e-8)  
interp_new = TMatrixInterpolator(tmfile_scuffNew, bspec, atol=1e-8)  
tmscuff_not_fixed = interp_old(omega)
tmscuff_bugfixed = interp_new(omega)

In [7]:
tmgen = TMatrixGenerator.sphere(medium, metal, radius)

In [8]:
tmgen_omega=tmgen(bspec,omega)

In [9]:
tmgen_omega.as_ndarray() # T-Matrix from generator

array([[-0.07824951+0.25215549j,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        , -0.07824951+0.25215549j,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        , -0.07824951+0.25215549j,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        , -0.00083788-0.01420874j,  0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        , -0.00083788-0.01420874j,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        ,  0.        +0.j        , -0.00083788-0.01420874j]])

In [10]:
tmscuff_not_fixed.as_ndarray() # T-Matrix of not fixed version of Scuff-EM

array([[-4.70886671e-03+1.79440815e-02j,  1.60098226e-13+4.20092462e-13j,  2.26455777e-07+3.98932962e-07j, -2.13258227e-14-6.01930824e-14j,  7.66251504e-08-1.32539814e-08j,
        -7.76488937e-14+1.08532726e-13j],
       [-4.41531068e-13+9.81757019e-14j, -4.70582367e-03+1.79389513e-02j, -1.60702588e-13-4.19762769e-13j,  6.32372612e-08+4.71302619e-08j, -2.36039655e-14+1.00983241e-13j,
         7.76590838e-08-1.38092126e-08j],
       [-4.58579591e-07-7.49053314e-09j,  4.41175187e-13-9.87457125e-14j, -4.70886672e-03+1.79440815e-02j,  2.13790755e-14+1.31812722e-13j,  6.25535562e-08+4.61875295e-08j,
         4.59276044e-14-4.47400043e-14j],
       [ 4.93366461e-14-4.29235249e-14j, -6.94450011e-09-1.60278760e-08j, -7.79609985e-14+1.07845028e-13j, -3.03166095e-05-2.01474828e-03j, -2.39253108e-13-1.34944425e-13j,
         1.16328171e-08+6.36863105e-09j],
       [ 8.21008790e-09-1.53591010e-08j, -2.36580203e-14+1.00878213e-13j, -5.41800099e-09-1.65573897e-08j,  2.26431323e-13-1.55248143e-13j, 

In [11]:
tmscuff_bugfixed.as_ndarray() # T-Matrix of FIXED version of Scuff-EM

array([[-9.43033280e-02+3.59361755e-01j,  3.20658236e-12+8.41416904e-12j,  5.13892659e-06+8.57797510e-06j, -4.28949536e-13-1.20456867e-12j,  1.41239218e-06-2.46862181e-07j,
        -1.55480479e-12+2.17400571e-12j],
       [-8.84386127e-12+1.96169624e-12j, -9.42395846e-02+3.59248336e-01j, -3.22166474e-12-8.40342496e-12j,  1.16868119e-06+8.68484583e-07j, -4.72874254e-13+2.02296639e-12j,
         1.43308826e-06-2.58864855e-07j],
       [-9.99885677e-06+6.63229466e-08j,  8.83605486e-12-1.97841571e-12j, -9.43033282e-02+3.59361755e-01j,  4.27888728e-13+2.63976219e-12j,  1.15544356e-06+8.48824647e-07j,
         9.21093604e-13-8.94225073e-13j],
       [ 9.85267406e-13-8.59818272e-13j, -1.43866720e-06+2.57952214e-07j, -1.56233879e-12+2.15974773e-12j, -6.07143255e-04-4.03488631e-02j, -4.79139425e-12-2.70322093e-12j,
         1.52945653e-07+8.36194839e-08j],
       [-1.15196465e-06-8.46913318e-07j, -4.73514628e-13+2.02574744e-12j, -1.40806670e-06+2.47097086e-07j,  4.53481568e-12-3.10854698e-12j, 

In [12]:
#I play around with the different operations so everything is correct
tmscuffnew = tmscuff_bugfixed.as_ndarray()
tmscuffnew_mat = np.asmatrix(tmscuffnew)
tmscuffnew_dag = tmscuffnew_mat.getH()
tmscuffnew_mat[0:2,0:2], tmscuffnew_dag[0:2,0:2]

(matrix([[-9.43033280e-02+3.59361755e-01j,  3.20658236e-12+8.41416904e-12j],
         [-8.84386127e-12+1.96169624e-12j, -9.42395846e-02+3.59248336e-01j]]),
 matrix([[-9.43033280e-02-3.59361755e-01j, -8.84386127e-12-1.96169624e-12j],
         [ 3.20658236e-12-8.41416904e-12j, -9.42395846e-02-3.59248336e-01j]]))

In [13]:
A = np.matrix([[0,1],[2,3]])
B = np.matrix([[3,2],[1,0]])
AB = np.dot(A,B)
BA = np.dot(B,A)
AB,BA

(matrix([[1, 0],
         [9, 4]]), matrix([[4, 9],
         [0, 1]]))

In [14]:
sum1new = np.dot(tmscuffnew_dag,tmscuffnew) #is this the right order? Regarding the above, yes it is.
sum2new = 0.5*tmscuffnew.__add__(tmscuffnew_dag) 

In [15]:
powermatrix_scuffnew = sum1new.__add__(sum2new) #powermatrix for bugfixed scuff

In [16]:
#Power matrix for NOT bugfixed scuff
tmscuffold = tmscuff_not_fixed.as_ndarray()
tmscuffold_mat = np.asmatrix(tmscuffold)
tmscuffold_dag = tmscuffold_mat.getH()
sum1old = np.dot(tmscuffold_dag,tmscuffold) #is this the right order? Regarding the above, yes it is.
sum2old = 0.5*tmscuffold.__add__(tmscuffold_dag)

powermatrix_scuffold = sum1old.__add__(sum2old) 

In [17]:
#Power matrix for T matrix generator
tmscuffgen = tmgen_omega.as_ndarray()
tmscuffgen_mat = np.asmatrix(tmscuffgen)
tmscuffgen_dag = tmscuffgen_mat.getH()
sum1gen = np.dot(tmscuffgen_dag,tmscuffgen) #is this the right order? Regarding the above, yes it is.
sum2gen = 0.5*tmscuffgen.__add__(tmscuffgen_dag)

powermatrix_scuffgen = sum1gen.__add__(sum2gen) 

In [18]:
#Power matrix of generator is diagonal and all its eigenvalues are negative:
powermatrix_scuffgen[0,0], powermatrix_scuffgen[1,1], powermatrix_scuffgen[2,2], powermatrix_scuffgen[3,3], powermatrix_scuffgen[4,4], powermatrix_scuffgen[5,5]

((-0.008544129580638216+0j),
 (-0.008544129580638216+0j),
 (-0.008544129580638216+0j),
 (-0.0006352854997266949+0j),
 (-0.0006352854997266949+0j),
 (-0.0006352854997266949+0j))

In [19]:
powermatrix_scuffnew[0,0], powermatrix_scuffnew[1,1], powermatrix_scuffnew[2,2], powermatrix_scuffnew[3,3], powermatrix_scuffnew[4,4], powermatrix_scuffnew[5,5]

((0.04373066071852283+0j),
 (0.04370088172427582+0j),
 (0.043730660553373296+0j),
 (0.001021256123757952+0j),
 (0.0010213406041430792+0j),
 (0.0010212560750220145+0j))

In [20]:
powermatrix_scuffold[0,0], powermatrix_scuffold[1,1], powermatrix_scuffold[2,2], powermatrix_scuffold[3,3], powermatrix_scuffold[4,4], powermatrix_scuffold[5,5]

((-0.004364703222422517+0j),
 (-0.004361872918812184+0j),
 (-0.004364703232328262+0j),
 (-2.6256479824067564e-05+0j),
 (-2.6262303053141142e-05+0j),
 (-2.6256487319428013e-05+0j))

In [22]:
#So! There is a fixed factor between the N/M elements of Scuff new and old:
powermatrix_scuffnew[0,0]/powermatrix_scuffold[0,0], powermatrix_scuffnew[1,1]/powermatrix_scuffold[1,1], powermatrix_scuffnew[2,2]/powermatrix_scuffold[2,2], powermatrix_scuffnew[3,3]/powermatrix_scuffold[3,3], powermatrix_scuffnew[4,4]/powermatrix_scuffold[4,4], powermatrix_scuffnew[5,5]/powermatrix_scuffold[5,5]

((-10.019160178833705-0j),
 (-10.018834234211564-0j),
 (-10.019160118257586-0j),
 (-38.895393845668316-0j),
 (-38.88998623145963-0j),
 (-38.89538088616882-0j))

In [23]:
#And there is a fixed factor for new / generated as well! (hence also for old / generated)
powermatrix_scuffnew[0,0]/powermatrix_scuffgen[0,0], powermatrix_scuffnew[1,1]/powermatrix_scuffgen[1,1], powermatrix_scuffnew[2,2]/powermatrix_scuffgen[2,2], powermatrix_scuffnew[3,3]/powermatrix_scuffgen[3,3], powermatrix_scuffnew[4,4]/powermatrix_scuffgen[4,4], powermatrix_scuffnew[5,5]/powermatrix_scuffgen[5,5]

((-5.118211317583541-0j),
 (-5.11472600126595-0j),
 (-5.118211298254535-0j),
 (-1.6075545942687262-0j),
 (-1.6076875744566317-0j),
 (-1.60755451755371-0j))

In [21]:
powermatrix_scuffnew[5,5]/powermatrix_scuffgen[5,5], powermatrix_scuffnew[1,1]/powermatrix_scuffgen[1,1]/2

((-1.60755451755371-0j), (-2.557363000632975+0j))

In [22]:
#Let's also calculate the determinants
detnew = powermatrix_scuffnew[0,0]*powermatrix_scuffnew[1,1]*powermatrix_scuffnew[2,2]*powermatrix_scuffnew[3,3]*powermatrix_scuffnew[4,4]*powermatrix_scuffnew[5,5]
detold = powermatrix_scuffold[0,0]*powermatrix_scuffold[1,1]*powermatrix_scuffold[2,2]*powermatrix_scuffold[3,3]*powermatrix_scuffold[4,4]*powermatrix_scuffold[5,5]
detgen = powermatrix_scuffgen[0,0]*powermatrix_scuffgen[1,1]*powermatrix_scuffgen[2,2]*powermatrix_scuffgen[3,3]*powermatrix_scuffgen[4,4]*powermatrix_scuffgen[5,5]
detgen, detnew, detold

((1.599227713011597e-16-0j),
 (8.902299490051238e-14+0j),
 (1.504486797691096e-21-0j))

In [23]:
np.linalg.det(powermatrix_scuffgen), np.linalg.det(powermatrix_scuffnew), np.linalg.det(powermatrix_scuffold)

((1.5992277130116025e-16+0j),
 (8.902299447416401e-14+2.6128569373609595e-39j),
 (1.504486701753944e-21-2.892571608172536e-45j))

In [24]:
#We try to normalize the power matrix elements for each case with the corresponding determinants.

In [25]:
#T-matrix generator: 
powermatrix_scuffgen[0,0]/detgen, powermatrix_scuffgen[1,1]/detgen, powermatrix_scuffgen[2,2]/detgen, powermatrix_scuffgen[3,3]/detgen, powermatrix_scuffgen[4,4]/detgen, powermatrix_scuffgen[5,5]/detgen  

((-53426597795433.88+0j),
 (-53426597795433.88+0j),
 (-53426597795433.88+0j),
 (-3972451793812.1055+0j),
 (-3972451793812.1055+0j),
 (-3972451793812.1055+0j))

In [26]:
#Not bugfixed Scuff-EM: 
powermatrix_scuffold[0,0]/detold, powermatrix_scuffold[1,1]/detold, powermatrix_scuffold[2,2]/detold, powermatrix_scuffold[3,3]/detold, powermatrix_scuffold[4,4]/detold, powermatrix_scuffold[5,5]/detold  

((-2.901124309712079e+18+0j),
 (-2.899243067806416e+18+0j),
 (-2.901124316296214e+18+0j),
 (-1.7452117136795634e+16+0j),
 (-1.7455987711853198e+16+0j),
 (-1.7452122118800434e+16+0j))

In [27]:
#New Scuff-EM: 
powermatrix_scuffnew[0,0]/detnew, powermatrix_scuffnew[1,1]/detnew, powermatrix_scuffnew[2,2]/detnew, powermatrix_scuffnew[3,3]/detnew, powermatrix_scuffnew[4,4]/detnew, powermatrix_scuffnew[5,5]/detnew  

((491228819782.9563+0j),
 (490894310768.96173+0j),
 (491228817927.8229+0j),
 (11471823936.043226+0j),
 (11472772908.667904+0j),
 (11471823388.58987+0j))

In [28]:
#It might make more sense to renormalize the electric and magnetic parts of the power matrices separately:

In [29]:
#T-matrix generator:
detgen_el = powermatrix_scuffgen[0,0]*powermatrix_scuffgen[1,1]*powermatrix_scuffgen[2,2]
detgen_mag = powermatrix_scuffgen[3,3]*powermatrix_scuffgen[4,4]*powermatrix_scuffgen[5,5]
powermatrix_scuffgen[0,0]/detgen_el, powermatrix_scuffgen[1,1]/detgen_el, powermatrix_scuffgen[2,2]/detgen_el, powermatrix_scuffgen[3,3]/detgen_mag, powermatrix_scuffgen[4,4]/detgen_mag, powermatrix_scuffgen[5,5]/detgen_mag  

((13698.226641508763-0j),
 (13698.226641508763-0j),
 (13698.226641508763-0j),
 (2477776.413504498-0j),
 (2477776.413504498-0j),
 (2477776.413504498-0j))

In [30]:
#NOT bugfixed Scuff-EM:
detold_el = powermatrix_scuffold[0,0]*powermatrix_scuffold[1,1]*powermatrix_scuffold[2,2]
detold_mag = powermatrix_scuffold[3,3]*powermatrix_scuffold[4,4]*powermatrix_scuffold[5,5]
powermatrix_scuffold[0,0]/detold_el, powermatrix_scuffold[1,1]/detold_el, powermatrix_scuffold[2,2]/detold_el, powermatrix_scuffold[3,3]/detold_mag, powermatrix_scuffold[4,4]/detold_mag, powermatrix_scuffold[5,5]/detold_mag  

((52525.751093170744-0j),
 (52491.69062780147-0j),
 (52525.75121237855-0j),
 (1450208903.6280527-0j),
 (1450530534.6580672-0j),
 (1450209317.614944-0j))

In [31]:
#Bugfixed Scuff-EM:
detnew_el = powermatrix_scuffnew[0,0]*powermatrix_scuffnew[1,1]*powermatrix_scuffnew[2,2]
detnew_mag = powermatrix_scuffnew[3,3]*powermatrix_scuffnew[4,4]*powermatrix_scuffnew[5,5]
powermatrix_scuffnew[0,0]/detnew_el, powermatrix_scuffnew[1,1]/detnew_el, powermatrix_scuffnew[2,2]/detnew_el, powermatrix_scuffnew[3,3]/detnew_mag, powermatrix_scuffnew[4,4]/detnew_mag, powermatrix_scuffnew[5,5]/detnew_mag  

((523.2675016470721+0j),
 (522.9111754524715+0j),
 (523.2674996709441+0j),
 (958726.5381294343+0j),
 (958805.8459399715+0j),
 (958726.4923775065+0j))